In [50]:
# Sam Brown
# sam_brown@mines.edu
# June 26 2025
# Goal: Use new inter-event features calculations to predict time since for events. Not entirely practical but useful for understanding of data

# Directory
import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

# Imports
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

df = pd.read_csv("/Users/sambrown04/Documents/SURF/Preproc_data/upd10-18.csv")
df = df[508:3000] # Avoid "dark spots" for now

In [82]:
# Features and target
X = df[['tide_deriv', 'tide_height', 'A_diurn', 'A_semidiurn','disp_standardized', 'r2_standardized', 'slope_standardized']]
y = df['time_since']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = .2, random_state = 42)

# Standardize
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

# Pytorch tensors 
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

In [84]:
# Neural Net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 24)
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12,1)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x) # Regressing
        return x

In [86]:
model = Net()
criterion = nn.MSELoss() # Loss for regression
optimizer = optim.Adam(model.parameters(), lr = .01)

# Training loop
epochs = 400
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # Clears grad

    # Predictions and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backprop
    loss.backward()

    # Update params
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [20/400], Loss: 0.1437
Epoch [40/400], Loss: 0.0537
Epoch [60/400], Loss: 0.0382
Epoch [80/400], Loss: 0.0303
Epoch [100/400], Loss: 0.0252
Epoch [120/400], Loss: 0.0196
Epoch [140/400], Loss: 0.0152
Epoch [160/400], Loss: 0.0124
Epoch [180/400], Loss: 0.0105
Epoch [200/400], Loss: 0.0092
Epoch [220/400], Loss: 0.0083
Epoch [240/400], Loss: 0.0077
Epoch [260/400], Loss: 0.0073
Epoch [280/400], Loss: 0.0070
Epoch [300/400], Loss: 0.0067
Epoch [320/400], Loss: 0.0065
Epoch [340/400], Loss: 0.0063
Epoch [360/400], Loss: 0.0061
Epoch [380/400], Loss: 0.0059
Epoch [400/400], Loss: 0.0057


In [88]:
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor)
    y_pred = y_scaler.inverse_transform(y_pred_scaled.numpy())
    y_test_orig = y_scaler.inverse_transform(y_test_tensor.numpy())

r2 = r2_score(y_test_orig, y_pred)
print(r2)

0.9495849013328552
